In [ ]:
from scipy.integrate import ode
from scipy.integrate import odeint
import scipy
import numpy as np
import matplotlib.pyplot as plt
import control
import copy

In [ ]:
#def parameters

ts = 1e-4 # Samplefrequency
t_end = 0.005 
steps = int(1 / ts) # #Step
step_length = t_end/steps
f0 = 50
V_eff = 230 * np.sqrt(2)

vi1 = 230
vi2 = 230

R = 0.4
L = 2.3e-3
C = 10e-6
LT = 2.3e-3
RLoad = 14

t = np.linspace(0, t_end, steps)


num_episodes = 1  # number of simulation episodes

In [ ]:
step_length

In [ ]:
#def varaibles

R1 = R
L1 = L
C1 = C
R2 = R
L2 = L
C2 = C
LT1 = LT
LT2 = LT
RT1 = R
RT2 = R

In [ ]:
A1 = np.array([[-R1/L1, -1/L1, 0],
               [1/C1, 0, -1/C1],
               [0, 1/LT1, -RLoad/LT1 - RT1/LT1]])

A2 = np.array([[-R2/L2, -1/L2, 0],
               [1/C2, 0, -1/C2],
               [0, 1/LT2, -RLoad/LT2 - RT2/LT2]])

AT1 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RLoad/LT1]])

AT2 = np.array([[0, 0, 0],
                [0, 0, 0],
                [0, 0, -RLoad/LT2]])

In [ ]:
A = np.block([[A1, AT1],
              [AT2, A2]])

In [ ]:
# x = np.array([i1, v1, iT1, i2, v2, iT2])
zeros = np.zeros((3,1))

In [ ]:
B1 = np.array([[1/L1],
              [0],
              [0]])

B2 = np.array([[1/L2],
              [0],
              [0]])

B = np.block([[B1, zeros],
             [zeros, B2]])

In [ ]:
C1 = np.array([[1, 1, 1]])
C2 = np.array([[1, 1, 1]])
C = np.block([C1, C2])

In [ ]:
u = np.array([vi1, vi2])[:,None] * np.ones((2,len(t)))

In [ ]:
i10 = 0
v10 = 0
iT10 = 0
i20 = 0
v20 = 0
iT20 = 0
t0 = 0

x0 = np.array([i10, v10, iT10, i20, v20, iT20])

# Continous solution

In [ ]:
sys = control.ss(A, B, C, 0)

In [ ]:
control.step_info(sys)

In [ ]:
T, yout, xout = control.forced_response(sys, T=t, U=u, X0=x0, return_x=True, squeeze=True)

In [ ]:
plt.plot(t, xout[1], label='v1')
# plt.plot(t,result[:steps,0], label = 'i1')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
# plt.title('{}'.format())
plt.legend()
plt.grid()
# plt.xlim([0, 0.00005])
# plt.ylim([49.25,50.1])
plt.show()

In [ ]:
T, yout = control.step_response(sys, T=t, X0=x0, squeeze=True)

In [ ]:
plt.plot(t, yout[1,:], label='v1')
# plt.plot(t,result[:steps,0], label = 'i1')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
# plt.title('{}'.format())
plt.legend()
plt.grid()
# plt.xlim([0, 0.00005])
# plt.ylim([49.25,50.1])
plt.show()

# Calculating the State Transition Matrix

In [ ]:
# AST Lecture 3 Slide 31. Equ. 3.32
def integrand(t, x):
    return scipy.linalg.expm(A*t) @ B @ u

### Single Evaluation at time t

In [ ]:
def x_t(t, u, x0, A, B):
    
    def integrand(t):
        return scipy.linalg.expm(A*t) @ B @ u
    
    stm_t = scipy.linalg.expm(A*t)
    
    x_h = stm_t @ x0
    print(x_h)
    x_p, err = scipy.integrate.quad_vec(integrand, 0, t)
    return x_h + x_p

In [ ]:
x_1 = x_t(t[-1], u, x0, A, B)

### Evaluation for a vector t

Super slow due to the for loop

In [ ]:
def x_dot(t, u, x0, A, B):
    
    def integrand(t):
        return scipy.linalg.expm(A*t) @ B @ u
    
    smt = np.zeros((len(t), A.shape[0], A.shape[1]))
    res =  np.zeros((len(t), A.shape[0]))
    x_p_cum = 0
    
    for i, t_ in enumerate(t):
        if i+1 == len(t):
            break
        stm_t = scipy.linalg.expm(A*t[i])
    
        x_h = stm_t @ x0
        x_p, err = scipy.integrate.quad_vec(integrand, t[i], t[i+1])
        
        x_p_cum += x_p 
        
        res[i] = x_h + x_p
    return res

In [ ]:
res = x_dot(t, u, x0, A, B)

### Speed up version using odeint

In [ ]:
%%time
x_h = scipy.sparse.linalg.expm_multiply(A, x0, start=0, stop=t_end, num=steps)
# As x0 = 0 the result is allways 0

x_p = odeint(integrand, x0, t, tfirst=True)
# Should be check if there is a simpler solution, bc x isn't present in the function

res = x_h + x_p

In [ ]:
plt.plot(t, res[:,1], label='v1')
# plt.plot(t,result[:steps,0], label = 'i1')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
# plt.title('{}'.format())
plt.legend()
plt.grid()
# plt.xlim([0, 0.00005])
# plt.ylim([49.25,50.1])
plt.show()

# Discret version

In [ ]:
t_ = np.arange(0, t_end, ts) 

In [ ]:
u_ = np.array([vi1, vi2])[:,None] * np.ones((2,len(t_)))

In [ ]:
A_d = scipy.linalg.expm(A*ts)

In [ ]:
A_d

In [ ]:
A_inv = scipy.linalg.inv(A)

In [ ]:
B_d = A_inv @ (A - np.eye(A.shape[0])) @ B

In [ ]:
B_d

In [ ]:
C_d = copy.copy(C)

In [ ]:
sys_d = control.ss(A_d, B_d, C_d, 0, step_length)

In [ ]:
t.shape

In [ ]:
control.step_info(sys_d)

In [ ]:
T, yout, xout = control.forced_response(sys_d, T=t_, U=u_, X0=x0, return_x=True, squeeze=True)

In [ ]:
steps

In [ ]:
xout[:,-1]

In [ ]:

plt.step(T, xout[1], label='v1')
# plt.plot(t,result[:steps,0], label = 'i1')
plt.xlabel(r'$t\,/\,\mathrm{s}$')
plt.ylabel('$v_{\mathrm{1}}\,/\,\mathrm{V}$')
# plt.title('{}'.format())
plt.legend()
plt.grid()
# plt.xlim([0, 0.00005])
# plt.ylim([49.25,50.1])
plt.show()